In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
var defaultVarsBE = await Workspace.Query<IfrsVariable>().ToArrayAsync();
await Workspace.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync()

In [0]:
var yieldCurve_Test = @"
@@YieldCurve
Currency,Year,Month,Scenario,Values0,Values1,Values2,Values3
CHF,2021,6,,0.003,0.003,0.003,0.003
CHF,2021,6,MTUP10pct,0.0033,0.0033,0.0033,0.0033";

In [0]:
var cashflows_BestEstimate = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2021,6,
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-3.3
DT1.1,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";
var file = "tempScenario.csv";


In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);

In [0]:
var yieldCurveLog = await Import.FromString(yieldCurve_Test).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
await Workspace.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync()

# Debug

In [0]:
var extension = System.IO.Path.GetExtension(file);
var stream = await Project.FileStorage.ReadAsync(file);
var dataSet = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

// begin cashflow importer

    var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Cashflow};
    await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, args);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var workspace = Workspace.CreateNew();
    
// begin Parse Cashflow 
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
    
    Activity.Start();
    var parsingStorage = new ParsingStorage(args, DataSource, workspace);
    await parsingStorage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();
    
    var importLog = await Import.FromDataSet(dataSet)
        .WithType<RawVariable> ( (dataset, datarow) => {
            var aocType = datarow.Field<string>(nameof(RawVariable.AocType));
            var novelty = datarow.Field<string>(nameof(RawVariable.Novelty));
            var dataNode = datarow.Field<string>(nameof(DataNode));
            
            if(!parsingStorage.DataNodeDataBySystemName.TryGetValue(dataNode, out var dataNodeData)) {
                ApplicationMessage.Log(Error.InvalidDataNode, dataNode);
                return null;
            }
            
            // Error if AocType is not present in the mapping
            if(!parsingStorage.AocTypeMap.Contains(new AocStep(aocType, novelty))) {
                ApplicationMessage.Log(Error.AocTypeMapNotFound, aocType, novelty);
                return null;
            }
                        
            // Filter out cashflows for DataNode that were created in the past and are still active and come with AocType = BOPI
            if(dataNodeData.Year < args.Year && aocType == AocTypes.BOP && novelty == Novelties.I) {
                ApplicationMessage.Log(Warning.ActiveDataNodeWithCashflowBOPI, dataNode);
                return null;
            }
                        
            var amountTypeFromFile = datarow.Field<string>(nameof(RawVariable.AmountType));
            var isEstimateType = parsingStorage.EstimateType.ContainsKey(amountTypeFromFile);
            var amountType = isEstimateType ? null : amountTypeFromFile;
            var estimateType = isEstimateType ? amountTypeFromFile : EstimateTypes.BE;
            
            var values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(RawVariable.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).Prune();
            
            // Filter out empty raw variables for AocType != CL
            if(values.Length == 0 && aocType != AocTypes.CL) return null;  //TODO: extend this check for all mandatory step and not just for CL
                
            var item = new RawVariable {
                DataNode = dataNode,
                AocType = aocType,
                Novelty = novelty,
                AmountType = amountType,
                EstimateType = estimateType,
                AccidentYear = dataset.Tables[ImportFormats.Cashflow].Columns.Where(x => x.ColumnName == nameof(RawVariable.AccidentYear)).Count()>0?
                            (Int32.TryParse((datarow.Field<string>(nameof(RawVariable.AccidentYear))), out var accidentYear)? accidentYear : (int?)null) : (int?)null, 
                Partition = parsingStorage.TargetPartitionByReportingNodeAndPeriod.Id,
                Values = Multiply(GetSign((aocType, amountType, estimateType, dataNodeData.IsReinsurance), parsingStorage.HierarchyCache), values)
            };
            return item;
        }, ImportFormats.Cashflow
    ).WithTarget(workspace).ExecuteAsync();
    
    await ValidateForDataNodeStateActiveAsync<RawVariable>(workspace, parsingStorage.DataNodeDataBySystemName);
    // put a concat and initialize the fisrt storage by itself, refacotr calculation of IfrsVars                                    
    /*foreach (var partition in calculatingPartitions)
    {
        var partition = calculatingPartitions[1];
        args = new ImportArgs(partition.ReportingNode, partition.Year, partition.Month, Periodicity.Quarterly, partition.Scenario, ImportFormats.Cashflow);
        var storage = new ImportStorage(args, DataSource, workspace);
        await storage.InitializeAsync();
        if(Activity.HasErrors()) return Activity.Finish();
   
        var universe = Scopes.ForStorage(storage).ToScope<IModel>();
        var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
        var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
        if(Activity.HasErrors()) return Activity.Finish();
    
        await workspace.UpdateAsync<IfrsVariable>(ivs);
        await CommitToDatabase<IfrsVariable>(workspace,
                                         storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
        await CommitToDatabase<RawVariable>(workspace, 
                                        storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
        workspace = Workspace.CreateNew();
        workspace.Initialize(x => x.FromSource(DataSource));
    }*/
return Activity.Finish();



In [0]:
//await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
//await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
await workspace.Query<RawVariable>().ToArrayAsync()

In [0]:
        var partition = calculatingPartitions[0];
        args = new ImportArgs(partition.ReportingNode, partition.Year, partition.Month, Periodicity.Quarterly, partition.Scenario, ImportFormats.Cashflow);
        var storage = new ImportStorage(args, DataSource, workspace);
        await storage.InitializeAsync();
        if(Activity.HasErrors()) return Activity.Finish();
   
        var universe = Scopes.ForStorage(storage).ToScope<IModel>();
        var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
        var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
        if(Activity.HasErrors()) return Activity.Finish();
    
        await workspace.UpdateAsync<IfrsVariable>(ivs);
        await CommitToDatabase<IfrsVariable>(workspace,
                                         storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
        await CommitToDatabase<RawVariable>(workspace, 
                                        storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
        //workspace = Workspace.CreateNew();
        //workspace.Initialize(x => x.FromSource(DataSource));
        workspace.Partition.GetCurrentPartitions()

In [0]:
var workspace1 = Workspace.CreateNew();
workspace1.Initialize(x => x.FromSource(DataSource));
 await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
 //check rawvars after workspace reset
await workspace1.Query<RawVariable>().ToArrayAsync()

In [0]:
// debug importstorage
int periodicityInMonths = 3;
var hierarchyCache = workspace.ToHierarchicalDimensionCache();
var args = argsScenario;
string ImportFormat => args.ImportFormat;
(int Year, int Month) CurrentReportingPeriod = (args.Year, args.Month);
(int Year, int Month) PreviousReportingPeriod => (args.Year - 1, MonthInAYear); // YTD Logic
//Dimensions
        var estimateTypes = (await workspace1.Query<EstimateType>().ToArrayAsync());
        
        var EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
        var AmountTypeDimension       = (await workspace1.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        var NoveltyDimension          = (await workspace1.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
        var ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}}; //TODO move this logic
        
        //Hierarchy Cache
        await hierarchyCache.InitializeAsync<AmountType>();
        
        //EstimateType to load and to update
        var EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow,}
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
        //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
        var ProjectionConfiguration = (await workspace1.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % periodicityInMonths == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
        //Get Partitions
        var PartitionByRn = (await DataSource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
        var TargetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        var DefaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        //Set Partitions
        await DataSource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        await workspace1.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        await workspace1.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
        //Get data from workspace1 (result of parsing)
        var parsedRawVariables = await workspace1.Query<RawVariable>().ToArrayAsync();
        var parsedIfrsVariables = await workspace1.Query<IfrsVariable>().ToArrayAsync();
        
        //DataNodes
        var DataNodeDataBySystemName = await workspace1.LoadDataNodesAsync(args);
        
        //Accident Years
        var AccidentYearsByDataNode = (IDictionary<string, ICollection<int?>>)
            (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => new {x.DataNode, x.AccidentYear}) : parsedIfrsVariables.Select(x => new {x.DataNode, x.AccidentYear}))
            .ToDictionaryGrouped(x => x.DataNode, x => (ICollection<int?>)x.Select(y => y.AccidentYear).ToHashSet());
        
        // Import Scopes and Data Node relationship parameters
        var InterDataNodeParametersByGoc = await workspace1.LoadInterDataNodeParametersAsync(args);
        
        var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
        var primaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();
        
        var primaryScope = primaryScopeFromParsedVariables.Concat(primaryScopeFromLinkedReinsurance).ToHashSet();
        var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
        var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
        
        var DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
        // Parameters
        var PartnerRating = await workspace1.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
        var CreditDefaultRates = await workspace1.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
        var SingleDataNodeParametersByGoc = await workspace1.LoadSingleDataNodeParametersAsync(args);

        var LockedInYieldCurve = await workspace1.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
        var CurrentYieldCurve = await workspace1.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
        var AocConfigurationByAocStep = await DataSource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
        var aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        
        //Previous Period
        var openingRawVariables = Enumerable.Empty<RawVariable>();
        var openingIfrsVariables = Enumerable.Empty<IfrsVariable>();

        if(allImportScopes.Select(dn => DataNodeDataBySystemName[dn]).Any(dnd => dnd.Year < args.Year)) {
            var PreviousPeriodPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                           p.Year == PreviousReportingPeriod.Year &&
                                                                                                           p.Month == PreviousReportingPeriod.Month &&
                                                                                                           p.Scenario == null).ToArrayAsync()).Single().Id;
            
            await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(PreviousPeriodPartition);
        
            //Perform queries to previous Period
            var openingRawVariables = (await DataSource.Query<RawVariable>()
                                   .Where(rv => rv.Partition == PreviousPeriodPartition && rv.AocType == AocTypes.CL)
                                   .Where(v => primaryScope.Contains(v.DataNode))
                                   .ToArrayAsync())
                .Select(rv => rv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Values = rv.Values.Skip(MonthInAYear).ToArray(), Partition = TargetPartition});
            
            var openingIfrsVariables = (await DataSource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == PreviousPeriodPartition && iv.AocType == AocTypes.EOP)
                                    .Where(v => allImportScopes.Contains(v.DataNode))
                                    .ToArrayAsync())
                .Select(iv => iv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Partition = TargetPartition});
            
            // TODO: print error if 
            //openingRawVariables.Select(x => x.DataNode).ToHashSet() != dataNodesWithPreviousPeriod
        }

In [0]:
TargetPartition

In [0]:
        var partition = calculatingPartitions[1];
        args = new ImportArgs(partition.ReportingNode, partition.Year, partition.Month, Periodicity.Quarterly, partition.Scenario, ImportFormats.Cashflow);
        var storage = new ImportStorage(args, DataSource, workspace1);
        await storage.InitializeAsync();
        if(Activity.HasErrors()) return Activity.Finish();
   
        /*var universe = Scopes.ForStorage(storage).ToScope<IModel>();
        var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
        var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
        if(Activity.HasErrors()) return Activity.Finish();
    
        await workspace1.UpdateAsync<IfrsVariable>(ivs);
        await CommitToDatabase<IfrsVariable>(workspace1,
                                         storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
        await CommitToDatabase<RawVariable>(workspace1, 
                                        storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
        workspace1.Partition.GetCurrentPartitions()*/
        storage.GetRawVariables("CH")

In [0]:
storage.GetRawVariables("CH")

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
await workspace.Query<RawVariable>().ToArrayAsync()

In [0]:
//await Import.FromString(cashflows_BestEstimate).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenario = new ImportArgs ("CH", 2021, 6, Periodicity.Quarterly, "MTUP10pct", ImportFormats.Cashflow);

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ifrsVars = await Workspace.Query<IfrsVariable>().ToArrayAsync();
ifrsVars.Length.Should().NotBe(0);

In [0]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenario);
var ifrsVars = await Workspace.Query<IfrsVariable>().ToArrayAsync();
ifrsVars.Length.Should().NotBe(0);

In [0]:
//Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
await Workspace.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync()

In [0]:
var extension = System.IO.Path.GetExtension(file);
var stream = await Project.FileStorage.ReadAsync(file);
var mainTab = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet.Tables[Main];